In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


/Users/tianji/mambaforge/envs/myenv/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Create a pipeline for Alu overlapping with Exon 

In [4]:
from HelperFunctions import calculate_overlaps

# Define standard BED column names (you can trim this list if your BED file has fewer columns)
cass_columns = [
    'chrom', 'whole_exon_start', 'whole_exon_end', 'name', 'score', 'strand',
    'thickStart', 'thickEnd', 'itemRgb', 'blockCount',
    'blockSizes', 'blockStarts'
]

alu_columns = [
    'chrom', 'start', 'end', 'name', 'score', 'strand'
]

# Helper function to extract the middle integer from a comma-separated string
def get_middle_int(s):
    parts = list(map(int, s.strip(',').split(',')))
    mid_idx = len(parts) // 2  # middle index (2nd block if 3 blocks)
    return parts[mid_idx]

# Extract helper functions
def get_first_int(s):
    return int(s.strip(',').split(',')[0])

def get_third_int(s):
    return int(s.strip(',').split(',')[2])

# Read in Alu elements reference bed file
alu_ref_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/CLIP Data/Alu-CassExon Reference Data/rmsk.Alu.uniqName2.bed', sep='\t', header=None, names=alu_columns)
alu_ref_df['alu_index'] = range(len(alu_ref_df))

# Read in cassette exon reference bed file
cass_exon_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/CLIP Data/Alu-CassExon Reference Data/Hs.seq.all.cass.chrom.can.bed', sep='\t', header=None, names=cass_columns)
print(f"Total number of cass exons from input file: {len(cass_exon_df)}")
cass_exon_df = cass_exon_df[cass_exon_df['blockCount'] == 3]
print(f"Total number of cass exons from input file with 3 exons: {len(cass_exon_df)}")
cass_exon_df['index'] = range(len(cass_exon_df))

# Apply the helper functions
cass_exon_df['mid_exon_start'] = cass_exon_df['whole_exon_start'] + cass_exon_df['blockStarts'].apply(get_middle_int)
cass_exon_df['mid_exon_end'] = cass_exon_df['mid_exon_start'] + cass_exon_df['blockSizes'].apply(get_middle_int)

# Calculate intron positions
cass_exon_df['ui_intron_start'] = cass_exon_df['whole_exon_start'] + cass_exon_df['blockSizes'].apply(get_first_int) + 1
cass_exon_df['ui_intron_end'] = cass_exon_df['mid_exon_start'] - 1

cass_exon_df['di_intron_start'] = cass_exon_df['mid_exon_end'] + 1
cass_exon_df['di_intron_end'] = cass_exon_df['whole_exon_start'] + cass_exon_df['blockStarts'].apply(get_third_int) - 1

# Exon dataframe
exon_df = cass_exon_df[['index','chrom', 'mid_exon_start', 'mid_exon_end', 'name', 'strand']].copy()
exon_df.rename(columns={'mid_exon_start': 'start', 'mid_exon_end': 'end'}, inplace=True)

# Upstream intron dataframe
first_intron_df = cass_exon_df[['index','chrom', 'ui_intron_start', 'ui_intron_end', 'name', 'strand']].copy()
first_intron_df.rename(columns={'ui_intron_start': 'start', 'ui_intron_end': 'end'}, inplace=True)

# Downstream intron dataframe
second_intron_df = cass_exon_df[['index','chrom', 'di_intron_start', 'di_intron_end', 'name', 'strand']].copy()
second_intron_df.rename(columns={'di_intron_start': 'start', 'di_intron_end': 'end'}, inplace=True)

ui_df = pd.concat([first_intron_df[first_intron_df['strand'] == '+'], second_intron_df[second_intron_df['strand'] == '-']])
di_df = pd.concat([first_intron_df[first_intron_df['strand'] == '-'], second_intron_df[second_intron_df['strand'] == '+']])

# Get unique chromosomes from ui_df/exon_df/di_df
chroms = set(first_intron_df['chrom'].unique())

# Subset alu_ref_df to rows where 'chrom' is in ui_chroms
alu_ref_subset = alu_ref_df[alu_ref_df['chrom'].isin(chroms)]


ku_asalu_tag_file = '/Users/tianji/Desktop/Alu Project/CLIP Data/Ku Map Alu Data/rmsk.reverse.Alu.uniqName.Ku.Bound.bed'
ku_ssalu_tag_file = '/Users/tianji/Desktop/Alu Project/CLIP Data/Ku Map Alu Data/rmsk.Alu.uniqName.Ku.Bound.bed'


Total number of cass exons from input file: 85522
Total number of cass exons from input file with 3 exons: 42761


In [5]:
# Read the files
df1 = pd.read_csv(ku_asalu_tag_file)
df2 = pd.read_csv(ku_ssalu_tag_file)

# Print number of rows (excluding header)
print(f"Total samples in 1: {len(df1)}")
print(f"Total samples in 2: {len(df2)}")

Total samples in 1: 6438
Total samples in 2: 95


In [6]:
# Read BED file with specified column names
ku_mapped_asalu_df = pd.read_csv(
    ku_asalu_tag_file,
    sep='\t',
    header=None,
    names=['chrom', 'start', 'end', 'name', 'score', 'strand']
)

ku_mapped_ssalu_df = pd.read_csv(
    ku_ssalu_tag_file,
    sep='\t',
    header=None,
    names=['chrom', 'start', 'end', 'name', 'score', 'strand']
)

# Count occurrences of each unique name in BED file
ku_asAlu_count_summary = ku_mapped_asalu_df['name'].value_counts().reset_index()
ku_asAlu_count_summary.columns = ['name', 'count']
ku_ssAlu_count_summary = ku_mapped_ssalu_df['name'].value_counts().reset_index()
ku_ssAlu_count_summary.columns = ['name', 'count']

In [7]:
ku_asAlu_count_summary

,name,count
0,AluJb#364,1
1,AluSx#704845,1
2,AluYg6#707701,1
3,AluSg#707684,1
4,AluJo#707136,1
...,...,...
6434,AluSx1#327405,1
6435,AluSq2#327161,1
6436,AluY#326928,1
6437,AluSx1#326915,1


In [8]:
print(ku_asAlu_count_summary['name'].str.contains('AluJb#402374', na=False).any())
print(ku_ssAlu_count_summary['name'].str.contains('AluJb#402374', na=False).any())
# Get the sets of names from each DataFrame
as_names = set(ku_asAlu_count_summary['name'])
ss_names = set(ku_ssAlu_count_summary['name'])

# Find intersection
common_names = as_names.intersection(ss_names)

# Count how many are the same
print("Number of shared names:", len(common_names))

# If you want to see what they are:
print("Shared names:", common_names)

False
True
Number of shared names: 1
Shared names: {'AluJb#1112578'}


In [9]:
exon_cass_df = calculate_overlaps(cass_exon_df=cass_exon_df, exon_coordinates = exon_df, alu_coordinates = alu_ref_subset, ku_ssalu_bed = ku_ssalu_tag_file, ku_asalu_bed = ku_asalu_tag_file, overlap_type='exon', min_overlap=10)
exon_cass_df.rename(columns = {
    'start_alu': 'exon_alu_start',
    'end_alu': 'exon_alu_end',
    'name_alu': 'exon_alu_name',
    'alu_index': 'exon_alu_index',
    'alu_orient': 'exon_alu_orient',
    'ku_tag_num': 'exon_alu_ku_tag_num'
}, inplace = True)

Total number of ssAlu with Ku tags: 96
Total number of asAlu with Ku tags: 6439
-----------------------------------------
Processing + strand in chr1
-----------------------------------------
Merging 2293 exon coordinates with 48889 ssalu coordinates
Merging 2293 exon coordinates with 49057 asalu coordinates
ssAlu and asAlu results merged! Total samples: 142. Total unique cass exons: 139
Total number of Ku bind asAlu is 5; ssAlu is 0
  Collected 139 overlaps for chr1 + 

-----------------------------------------
Processing + strand in chr2
-----------------------------------------
Merging 1661 exon coordinates with 40155 ssalu coordinates
Merging 1661 exon coordinates with 40041 asalu coordinates
ssAlu and asAlu results merged! Total samples: 107. Total unique cass exons: 107
Total number of Ku bind asAlu is 5; ssAlu is 0
  Collected 107 overlaps for chr2 + 

-----------------------------------------
Processing + strand in chr3
-----------------------------------------
Merging 1296 exo

In [10]:
ui_cass_df = calculate_overlaps(cass_exon_df=cass_exon_df, exon_coordinates = ui_df, alu_coordinates = alu_ref_subset,  ku_ssalu_bed = ku_ssalu_tag_file, ku_asalu_bed = ku_asalu_tag_file, min_overlap=0)
ui_cass_df.rename(columns = {
    'start_alu': 'ui_alu_start',
    'end_alu': 'ui_alu_end',
    'name_alu': 'ui_alu_name',
    'alu_index': 'ui_alu_index',
    'dist_to_ui_start': 'dist_to_ui_start',
    'dist_to_ui_end':'dist_to_ui_end',
    'alu_orient': 'ui_alu_orient',
    'ku_tag_num': 'ui_alu_ku_tag_num'

}, inplace = True)

Total number of ssAlu with Ku tags: 96
Total number of asAlu with Ku tags: 6439
-----------------------------------------
Processing + strand in chr1
-----------------------------------------
Merging 2293 exon coordinates with 48889 ssalu coordinates
Merging 2293 exon coordinates with 49057 asalu coordinates
ssAlu and asAlu results merged! Total samples: 10706. Total unique cass exons: 1291
Total number of Ku bind asAlu is 228; ssAlu is 0
  Collected 1291 overlaps for chr1 + 

-----------------------------------------
Processing + strand in chr2
-----------------------------------------
Merging 1661 exon coordinates with 40155 ssalu coordinates
Merging 1661 exon coordinates with 40041 asalu coordinates
ssAlu and asAlu results merged! Total samples: 6208. Total unique cass exons: 968
Total number of Ku bind asAlu is 107; ssAlu is 0
  Collected 968 overlaps for chr2 + 

-----------------------------------------
Processing + strand in chr3
-----------------------------------------
Merging

In [11]:
di_cass_df = calculate_overlaps(cass_exon_df=cass_exon_df, exon_coordinates = di_df, alu_coordinates = alu_ref_subset, ku_ssalu_bed = ku_ssalu_tag_file, ku_asalu_bed = ku_asalu_tag_file,  min_overlap=0)
di_cass_df.rename(columns = {
    'start_alu': 'di_alu_start',
    'end_alu': 'di_alu_end',
    'name_alu': 'di_alu_name',
    'alu_index': 'di_alu_index',
    'dist_to_ui_start': 'dist_to_di_start',
    'dist_to_ui_end':'dist_to_di_end',
    'alu_orient': 'di_alu_orient',
    'ku_tag_num': 'di_alu_ku_tag_num'
}, inplace = True)

Total number of ssAlu with Ku tags: 96
Total number of asAlu with Ku tags: 6439
-----------------------------------------
Processing + strand in chr1
-----------------------------------------
Merging 2293 exon coordinates with 48889 ssalu coordinates
Merging 2293 exon coordinates with 49057 asalu coordinates
ssAlu and asAlu results merged! Total samples: 9152. Total unique cass exons: 1326
Total number of Ku bind asAlu is 141; ssAlu is 0
  Collected 1326 overlaps for chr1 + 

-----------------------------------------
Processing + strand in chr2
-----------------------------------------
Merging 1661 exon coordinates with 40155 ssalu coordinates
Merging 1661 exon coordinates with 40041 asalu coordinates
ssAlu and asAlu results merged! Total samples: 5897. Total unique cass exons: 1034
Total number of Ku bind asAlu is 83; ssAlu is 1
  Collected 1034 overlaps for chr2 + 

-----------------------------------------
Processing + strand in chr3
-----------------------------------------
Merging

#### Merge overlap results and create annotation columns for AluTypes

In [16]:
############################################################
### (1) Merge ui_alu_df, di_alu_df, exon_alu_df together ###
############################################################
# Copy the DataFrame
cass_alu_df = ui_cass_df.copy()

# Compute mid_exon_length
cass_alu_df['mid_exon_length'] = cass_alu_df['mid_exon_end'] - cass_alu_df['mid_exon_start']

# Select specific columns
cass_alu_df = cass_alu_df[[
    'name', 'chrom', 'strand', 'mid_exon_length',
    'ui_alu_start', 'ui_alu_end', 'ui_alu_name', 'alu_score',
    'ui_alu_index', 'dist_to_ui_start', 'dist_to_ui_end', 'ui_alu_orient', 'ui_alu_ku_tag_num'
]]

"""Merge di overlapping results"""
# Prepare di_cass_df columns for merge
di_cols = [
    'name', 'di_alu_start', 'di_alu_end', 'di_alu_name', 'alu_score',
    'di_alu_index', 'dist_to_di_start', 'dist_to_di_end', 'di_alu_orient', 'di_alu_ku_tag_num'
]
di_cass_subset = di_cass_df[di_cols]

# Merge on 'name'
cass_alu_df = cass_alu_df.merge(di_cass_subset, on='name', how='left')

"""Merge middle exon overlapping results"""
exon_cols = [
    'name', 'exon_alu_start', 'exon_alu_end', 'exon_alu_name', 'alu_score',
    'exon_alu_index', 'exon_alu_orient', 'exon_alu_ku_tag_num'
]
exon_cass_subset = exon_cass_df[exon_cols]

# Merge on 'name'
cass_alu_df = cass_alu_df.merge(exon_cass_subset, on='name', how='left')

"""Merge several columns from previous annotated merged AS analysis results"""
# Merge the 'alu_type', 'alu_subtype', 'alu_specific_subtype' columns from merged cass alternative splicing results file to cass_alu_df
cass_as_result = pd.read_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.Merged.DiffAlterSplicing.results.step1.csv")
# Define the columns to select and their new names
cass_as_result_cols = [
    'name', 
    {'alu_type': 'alu_type_old'}, 
    {'alu_subtype': 'alu_subtype_old'}, 
    {'alu_specific_subtype': 'alu_specific_subtype_old'},
    'sensitivity'
]

# First convert to proper rename format
rename_dict = {}
for item in cass_as_result_cols:
    if isinstance(item, dict):
        rename_dict.update(item)

# Select and rename columns
cass_as_result_subset = cass_as_result[list(rename_dict.keys()) + ['name', 'sensitivity']].rename(columns=rename_dict)

# Perform the merge
cass_alu_df = cass_alu_df.merge(cass_as_result_subset, on='name', how='left')

# Clean up
del cass_as_result, cass_as_result_subset

"""Add six columns indicating existence of asAlu, ssAlu in ui and di"""
# Initialize new columns with 0
cass_alu_df['all_asAlu_ui'] = 0
cass_alu_df['all_ssAlu_ui'] = 0
cass_alu_df['all_asAlu_exon'] = 0
cass_alu_df['all_ssAlu_exon'] = 0
cass_alu_df['all_asAlu_di'] = 0
cass_alu_df['all_ssAlu_di'] = 0
cass_alu_df['ku_asAlu_ui'] = 0
cass_alu_df['ku_ssAlu_ui'] = 0
cass_alu_df['ku_asAlu_exon'] = 0
cass_alu_df['ku_ssAlu_exon'] = 0
cass_alu_df['ku_asAlu_di'] = 0
cass_alu_df['ku_ssAlu_di'] = 0

# Update columns conditionally using vectorized operations
cass_alu_df['all_asAlu_ui'] = np.where(
    cass_alu_df['ui_alu_orient'].str.contains('as', na=False), 1, 0
)
cass_alu_df['all_ssAlu_ui'] = np.where(
    cass_alu_df['ui_alu_orient'].str.contains('ss', na=False), 1, 0
)
cass_alu_df['all_asAlu_exon'] = np.where(
    cass_alu_df['exon_alu_orient'].str.contains('as', na=False), 1, 0
)
cass_alu_df['all_ssAlu_exon'] = np.where(
    cass_alu_df['exon_alu_orient'].str.contains('ss', na=False), 1, 0
)
cass_alu_df['all_asAlu_di'] = np.where(
    cass_alu_df['di_alu_orient'].str.contains('as', na=False), 1, 0
)
cass_alu_df['all_ssAlu_di'] = np.where(
    cass_alu_df['di_alu_orient'].str.contains('ss', na=False), 1, 0
)
def sum_ku_tags_for_as(row, ku_tag_col, alu_orient_column, alu_orient):
    if alu_orient not in ['as', 'ss']:
        raise ValueError("'alu_orient' must be 'as' or 'ss'.")

    # Handle missing data
    if pd.isna(row[alu_orient_column]) or pd.isna(row[ku_tag_col]):
        return 0

    # Split both fields (use fallback delimiter if needed)
    raw_orients = str(row[alu_orient_column])
    raw_tags = str(row[ku_tag_col])

    orient_delim = ', ' if ', ' in raw_orients else ','
    tag_delim = ', ' if ', ' in raw_tags else ','

    orients = [o.strip() for o in raw_orients.split(orient_delim)]
    tag_strs = [t.strip() for t in raw_tags.split(tag_delim)]

    # Convert tags to ints
    ku_tags = [int(t) for t in tag_strs]

    if len(orients) != len(ku_tags):
        return 0

    # Now sum where orient matches
    total = sum(tag for orient, tag in zip(orients, ku_tags) if orient == alu_orient)
    return total


cass_alu_df['ku_asAlu_ui'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='ui_alu_ku_tag_num', alu_orient_column = 'ui_alu_orient',alu_orient='as'), axis=1)
cass_alu_df['ku_ssAlu_ui'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='ui_alu_ku_tag_num', alu_orient_column = 'ui_alu_orient',alu_orient='ss'), axis=1)
cass_alu_df['ku_asAlu_di'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='di_alu_ku_tag_num', alu_orient_column = 'di_alu_orient',alu_orient='as'), axis=1)
cass_alu_df['ku_ssAlu_di'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='di_alu_ku_tag_num', alu_orient_column = 'di_alu_orient',alu_orient='ss'), axis=1)
cass_alu_df['ku_asAlu_exon'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='exon_alu_ku_tag_num', alu_orient_column = 'exon_alu_orient',alu_orient='as'), axis=1)
cass_alu_df['ku_ssAlu_exon'] = cass_alu_df.apply(lambda row: sum_ku_tags_for_as(row, ku_tag_col='exon_alu_ku_tag_num', alu_orient_column = 'exon_alu_orient',alu_orient='ss'), axis=1)


In [17]:
cass_alu_df['ku_asAlu_di'].value_counts()

ku_asAlu_di
0    40747
1     1551
2      316
3       72
4       43
6       17
5        9
7        5
8        1
Name: count, dtype: int64

In [18]:
##############################################################
### (2) Annotate AluType using the six new alu tag columns ###
##############################################################
"""First define different conditions for asAlu, irAlu, ssAlu and other cassette exons:
    --asAlu: 2nd exon contains asAlu
    --irAlu: (ui has asAlu and di has ssAlu OR ui has ssAlu and di has asAlu) AND (2nd exon has no asAlu)
    --ssAlu: 2nd exon contains ssAlu but not asAlu AND Not (ui has asAlu and di has ssAlu OR ui has ssAlu and di has asAlu)
    --other:
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

# Simplify the name of cass_df_with_gtex_brainspan_studer_psi_alu
cass_df = cass_alu_df.copy()
del cass_alu_df
# Condition for asAlu
asAlu_condition = (cass_df['all_asAlu_exon'] > 0.0)

# Two pre-conditions for irAlu
irAlu_pre_condition1 = ((cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0))
irAlu_pre_condition2 = ((cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0))
# Condition for irAlu
irAlu_condition = ((irAlu_pre_condition1 | irAlu_pre_condition2) & 
                                   (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0))

# Condition for ssAlu
ssAlu_condition = ((cass_df['all_ssAlu_exon'] > 0.0) & 
                   (cass_df['all_asAlu_exon'] == 0.0))

# Condition for other cassette exons
other_samples_condition = ~(asAlu_condition | irAlu_condition | ssAlu_condition)

# Check for overlapping categories
overlap_mask = (
    (asAlu_condition & irAlu_condition) |
    (asAlu_condition & ssAlu_condition) |
    (irAlu_condition & ssAlu_condition)
)
print(f"Number of samples in multiple categories: {overlap_mask.sum()}")

"""Create a new column 'alu_type' after 'name' column to annotate the type of each cassette exon event"""
# Create alu_type column using priority: asAlu > irAlu > ssAlu > other
conditions = [
    asAlu_condition,
    irAlu_condition,
    ssAlu_condition,
    other_samples_condition
]

choices = ['asAlu', 'irAlu', 'ssAlu', 'other']

# Insert new column after 'name'
cass_df.insert(
    loc=cass_df.columns.get_loc('name') + 1,
    column='alu_type',
    value=np.select(conditions[:-1], choices[:-1], default='other')
)

# Verify distribution
print("\nCategory counts:")
print(cass_df['alu_type'].value_counts())


########################################################################
"""Define different conditions for asAlu subtypes"""
########################################################################
""" --(1) asAlu_not_flanked_by_irAlu: 2nd exon contains asAlu, but ui and di does not contain irAlu
    --asAlu_flanked_by_irAlu: 2nd exon contains asAlu and ui and di also contains irAlu pair
                                                            |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
        > (2) asAlu_flanked_by_irAlu_without_local_competition:          <=         <=        =>     (irAlu_pre_condition1)
                                                                    or   =>         <=        <=     (irAlu_pre_condition2)
                                                            --------------------------------------------------
        > (3) asAlu_flanked_by_irAlu_with_local_competition:           <=  =>       <=        =>     (subcondition1)
                                                                    or   <=         <=      =>  <=   (subcondition2)
                                                                    or =>  <=       <=        <=     (subcondition3)
                                                                    or   =>         <=      <=  =>   (subcondition4)                                                                                 
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

"""Now let's set up conditions to break asAlu type into 3 subtypes"""
# conditions for asAlu not flanked by irAlu
asAlu_not_flanked_by_irAlu_condition = ~(irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition

# Four types of Local Competing irAlu subconditions
subcondition1 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
subcondition2 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
subcondition3 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
subcondition4 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
# Local irAlu Competing conditions
local_irAlu_competing_conditions = subcondition1 | subcondition2 | subcondition3 | subcondition4

# conditions for asAlu flanked by non-local competing irAlu
asAlu_flanked_by_irAlu_without_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition & ~local_irAlu_competing_conditions

# conditions for asAlu flanked by local competing irAlu
asAlu_flanked_by_irAlu_with_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition & local_irAlu_competing_conditions

# First verify the conditions
total_asAlu = asAlu_condition.sum()
sum_subtypes = (asAlu_not_flanked_by_irAlu_condition | 
               asAlu_flanked_by_irAlu_without_local_competition |
               asAlu_flanked_by_irAlu_with_local_competition).sum()

condition1_met = (sum_subtypes == total_asAlu)
if condition1_met:
    print("Great! Sum of three asAlu subtypes equal to total asAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of three asAlu subtypes does not equal to total asAlu")

# Check for overlaps between subtypes
overlap1 = (asAlu_not_flanked_by_irAlu_condition & 
           asAlu_flanked_by_irAlu_without_local_competition).sum()
overlap2 = (asAlu_not_flanked_by_irAlu_condition & 
           asAlu_flanked_by_irAlu_with_local_competition).sum()
overlap3 = (asAlu_flanked_by_irAlu_without_local_competition & 
           asAlu_flanked_by_irAlu_with_local_competition).sum()
condition2_met = (overlap1 + overlap2 + overlap3) == 0
if condition2_met:
    print(f"Great! No overlapping samples between asAlu subtypes")
else:
    print(f"Oh no! Overlapping samples between asAlu subtypes are found")

"""Define different conditions for ssAlu subtypes:
    --(1) ssAlu_not_flanked_by_irAlu: 2nd exon contains ssAlu, but ui and di does not contain irAlu
    --ssAlu_flanked_by_irAlu: 2nd exon contains ssAlu and ui and di also contains irAlu pair
                                                            |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
        > (2) ssAlu_flanked_by_irAlu_without_local_competition:          <=         =>        =>     (irAlu_pre_condition1)
                                                                    or   =>         =>        <=     (irAlu_pre_condition2)
                                                            --------------------------------------------------
        > (3) ssAlu_flanked_by_irAlu_with_local_competition:           <=  =>       =>        =>     (subcondition1)
                                                                    or   <=         =>      =>  <=   (subcondition2)
                                                                    or =>  <=       =>        <=     (subcondition3)
                                                                    or   =>         =>      <=  =>   (subcondition4)                                                                                 
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

########################################################################
"""Now let's set up conditions to break ssAlu type into 3 subtypes"""
########################################################################
# conditions for asAlu not flanked by irAlu
ssAlu_not_flanked_by_irAlu_condition = ~(irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition

# Four types of Local Competing irAlu subconditions
subcondition1 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
subcondition2 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
subcondition3 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
subcondition4 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
# Local irAlu Competing conditions
local_irAlu_competing_conditions = subcondition1 | subcondition2 | subcondition3 | subcondition4

# conditions for asAlu flanked by non-local competing irAlu
ssAlu_flanked_by_irAlu_without_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition & ~local_irAlu_competing_conditions

# conditions for asAlu flanked by local competing irAlu
ssAlu_flanked_by_irAlu_with_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition & local_irAlu_competing_conditions

# First verify the conditions
total_ssAlu = ssAlu_condition.sum()
sum_subtypes = (ssAlu_not_flanked_by_irAlu_condition | 
               ssAlu_flanked_by_irAlu_without_local_competition |
               ssAlu_flanked_by_irAlu_with_local_competition).sum()

condition1_met = (sum_subtypes == total_ssAlu)
if condition1_met:
    print("Great! Sum of three ssAlu subtypes equal to total ssAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of three ssAlu subtypes does not equal to total ssAlu")

# Check for overlaps between subtypes
overlap1 = (ssAlu_not_flanked_by_irAlu_condition & 
           ssAlu_flanked_by_irAlu_without_local_competition).sum()
overlap2 = (ssAlu_not_flanked_by_irAlu_condition & 
           ssAlu_flanked_by_irAlu_with_local_competition).sum()
overlap3 = (ssAlu_flanked_by_irAlu_without_local_competition & 
           ssAlu_flanked_by_irAlu_with_local_competition).sum()
condition2_met = (overlap1 + overlap2 + overlap3) == 0
if condition2_met:
    print(f"Great! No overlapping samples between ssAlu subtypes")
else:
    print(f"Oh no! Overlapping samples between ssAlu subtypes are found")

########################################################################
"""Now let's set up conditions to break irAlu type into 2 subtypes"""
########################################################################
"""
    (1) irAlu without internal competing irAlu: "irAlu_without_intronic_irAlu_competition"
        |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                     <=                   =>     (irAlu_no_competition1)
      or             =>                   <=     (irAlu_no_competition2)
    ----------------------------------------------------------    
    (2) irAlu with internal competing irAlu   
        |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|                
    or             <=  =>                 <=            
    or             =>  <=                 <=   
    or             <=  =>                 =>            
    or             =>  <=                 =>    
    or               =>                 <=  =>     
    or               =>                 =>  <=     
    or               <=                 <=  =>     
    or               <=                 =>  <=    
"""
# For (1) 
irAlu_no_competition1 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
irAlu_no_competition2 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
irAlu_without_intronic_irAlu_competition = irAlu_no_competition1 | irAlu_no_competition2
# For (2)
ui_irAlu_condition = (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_ui'] > 0.0) 
di_irAlu_condition = (cass_df['all_asAlu_di'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0) 
irAlu_no_competition1 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & ui_irAlu_condition & ((cass_df['all_ssAlu_di'] > 0.0) | (cass_df['all_asAlu_di'] > 0.0))
irAlu_no_competition2 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & di_irAlu_condition & ((cass_df['all_ssAlu_ui'] > 0.0) | (cass_df['all_asAlu_ui'] > 0.0))
irAlu_with_intronic_irAlu_competition = irAlu_no_competition1 | irAlu_no_competition2 

# First verify the conditions
total_asAlu = irAlu_condition.sum()
sum_subtypes = (irAlu_without_intronic_irAlu_competition | 
               irAlu_with_intronic_irAlu_competition).sum()

condition1_met = (sum_subtypes == total_asAlu)
if condition1_met:
    print("Great! Sum of two irAlu subtypes equal to total irAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of two irAlu subtypes does not equal to total irAlu")

# Check for overlaps between subtypes
overlap = (irAlu_without_intronic_irAlu_competition & 
           irAlu_with_intronic_irAlu_competition).sum()
if overlap == 0:
    print(f"Great! No overlapping samples between subtypes")
else:
    print(f"Oh no! Overlapping samples between subtypes are found")

########################################################################
"""Now let's set up conditions to break other samples type into 2 subtypes"""
########################################################################
""" (1) other samples contain at least one Alu element in each of two flanking introns and in same orientations
    |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                 <=                    <=   
      or         =>                    => 
    (2) other samples contain at least one Alu element in only one flanking intron
    |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                <=                                  (other_with_asAlu_only_ui)
      or        =>                                  (other_with_ssAlu_only_ui)
      or                              <=            (other_with_asAlu_only_di)       
      or                              =>            (other_with_ssAlu_only_di)
      or       <= =>                                (other_with_irAlu_only_ui)
      or                             <= =>          (other_with_irAlu_only_di)
    (3) remaining other samples
    """
other_samples_with_alu_condition = other_samples_condition & (
    (cass_df['all_asAlu_ui'] > 0.0) |
    (cass_df['all_ssAlu_ui'] > 0.0) |
    (cass_df['all_asAlu_di'] > 0.0) |
    (cass_df['all_ssAlu_di'] > 0.0)
)

other_samples_with_alus_in_two_introns_condition = other_samples_condition & (
    ((cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0)) |
    ((cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0))
)

other_samples_with_alus_in_one_intron_condition = other_samples_with_alu_condition & ~other_samples_with_alus_in_two_introns_condition

other_samples_without_alu_condition = other_samples_condition & ~other_samples_with_alu_condition

combined = (
    other_samples_with_alus_in_two_introns_condition |
    other_samples_with_alus_in_one_intron_condition |
    other_samples_without_alu_condition
)
other_sample_condition_met = combined.equals(other_samples_condition)

if other_sample_condition_met:
    print("Great! Sum of two other samples subtypes equal to total other samples")
else:
    print("Oh No! There's non-resolved condition. Sum of two other samples subtypes does not equal to total asAlu")

other_sample_overlap1 = ((other_samples_with_alus_in_two_introns_condition & other_samples_with_alus_in_one_intron_condition).sum() == 0)
other_sample_overlap2 = ((other_samples_with_alus_in_two_introns_condition & other_samples_without_alu_condition).sum() == 0)
other_sample_overlap3 = ((other_samples_with_alus_in_one_intron_condition & other_samples_without_alu_condition).sum() == 0)
other_sample_on_overlap_condition_met = (
    other_sample_overlap1 and other_sample_overlap2 and other_sample_overlap3
)

if other_sample_on_overlap_condition_met:
    print(f"Great! No overlapping samples between subtypes")
else:
    print(f"Oh no! Overlapping samples between subtypes are found")

# If the two condition checks are satified, proceed annotating alu subtypes for asAlu
conditions = [
    other_samples_with_alus_in_one_intron_condition,
    other_samples_with_alus_in_two_introns_condition,
    other_samples_without_alu_condition,
    asAlu_not_flanked_by_irAlu_condition,
    asAlu_flanked_by_irAlu_without_local_competition,
    asAlu_flanked_by_irAlu_with_local_competition,
    ssAlu_not_flanked_by_irAlu_condition,
    ssAlu_flanked_by_irAlu_without_local_competition,
    ssAlu_flanked_by_irAlu_with_local_competition,
    irAlu_without_intronic_irAlu_competition,
    irAlu_with_intronic_irAlu_competition
]
    
choices = [
    'other_with_Alu_in_one_flanking_intron',
    'other_with_same_strand_Alu_in_two_flanking_introns',
    'other_without_Alu',
    'asAlu_no_flanking_irAlu',
    'asAlu_with_flanking_noncompete_irAlu',
    'asAlu_with_flanking_compete_irAlu',
    'ssAlu_no_flanking_irAlu',
    'ssAlu_with_flanking_noncompete_irAlu',
    'ssAlu_with_flanking_compete_irAlu',
    'irAlu_no_intronic_irAlu_compete',
    'irAlu_with_intronic_irAlu_compete'
]
    
cass_df.insert(
    loc=cass_df.columns.get_loc('alu_type') + 1,
    column='alu_subtype',
    value=np.select(conditions, choices, default='other')
)

# Verify results
print("\nSubtype distribution:")
print(cass_df['alu_subtype'].value_counts())

# Base unchanged types
unchanged_subtypes = [
    'ssAlu',
    'other_without_Alu',
    'asAlu_no_flanking_irAlu',
    'asAlu_with_flanking_noncompete_irAlu',
    'asAlu_with_flanking_compete_irAlu',
    'irAlu_no_intronic_irAlu_compete',
    'irAlu_with_intronic_irAlu_compete'
]

# Start with the existing 'alu_subtype' column
cass_df.insert(
    loc=cass_df.columns.get_loc('alu_subtype') + 1,
    column='alu_specific_subtype',
    value=cass_df['alu_subtype']
)

# === Split subtype: 'other_with_Alu_in_one_flanking_intron' ===

# Define sub-conditions
asAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0) &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
)

asAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0) &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
)

ssAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

ssAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

irAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

irAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
)

# Apply annotations
cass_df.loc[asAlu_only_ui, 'alu_specific_subtype'] = 'other_with_asAlu_only_ui'
cass_df.loc[asAlu_only_di, 'alu_specific_subtype'] = 'other_with_asAlu_only_di'
cass_df.loc[ssAlu_only_ui, 'alu_specific_subtype'] = 'other_with_ssAlu_only_ui'
cass_df.loc[ssAlu_only_di, 'alu_specific_subtype'] = 'other_with_ssAlu_only_di'
cass_df.loc[irAlu_only_ui, 'alu_specific_subtype'] = 'other_with_irAlu_only_ui'
cass_df.loc[irAlu_only_di, 'alu_specific_subtype'] = 'other_with_irAlu_only_di'

# === Split subtype: 'other_with_same_strand_Alu_in_two_flanking_introns' ===

asAlu_both = (
    (cass_df['alu_subtype'] == 'other_with_same_strand_Alu_in_two_flanking_introns') &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0)
)

ssAlu_both = (
    (cass_df['alu_subtype'] == 'other_with_same_strand_Alu_in_two_flanking_introns') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
)

cass_df.loc[asAlu_both, 'alu_specific_subtype'] = 'other_with_asAlu_in_both_ui_di'
cass_df.loc[ssAlu_both, 'alu_specific_subtype'] = 'other_with_ssAlu_in_both_ui_di'

cass_df['alu_specific_subtype'].value_counts()



Number of samples in multiple categories: 0

Category counts:
alu_type
other    24718
irAlu    14941
asAlu     2678
ssAlu      424
Name: count, dtype: int64
Great! Sum of three asAlu subtypes equal to total asAlu
Great! No overlapping samples between asAlu subtypes
Great! Sum of three ssAlu subtypes equal to total ssAlu
Great! No overlapping samples between ssAlu subtypes
Great! Sum of two irAlu subtypes equal to total irAlu
Great! No overlapping samples between subtypes
Great! Sum of two other samples subtypes equal to total other samples
Great! No overlapping samples between subtypes

Subtype distribution:
alu_subtype
irAlu_with_intronic_irAlu_compete                     13723
other_with_Alu_in_one_flanking_intron                 13665
other_without_Alu                                      9859
asAlu_no_flanking_irAlu                                1237
irAlu_no_intronic_irAlu_compete                        1218
other_with_same_strand_Alu_in_two_flanking_introns     1194
asAlu_with_f

alu_specific_subtype
irAlu_with_intronic_irAlu_compete       13723
other_without_Alu                        9859
other_with_irAlu_only_di                 3542
other_with_irAlu_only_ui                 3317
other_with_ssAlu_only_di                 1779
other_with_asAlu_only_di                 1758
other_with_asAlu_only_ui                 1702
other_with_ssAlu_only_ui                 1567
asAlu_no_flanking_irAlu                  1237
irAlu_no_intronic_irAlu_compete          1218
asAlu_with_flanking_noncompete_irAlu      881
other_with_asAlu_in_both_ui_di            668
asAlu_with_flanking_compete_irAlu         560
other_with_ssAlu_in_both_ui_di            526
ssAlu_no_flanking_irAlu                   237
ssAlu_with_flanking_noncompete_irAlu       99
ssAlu_with_flanking_compete_irAlu          88
Name: count, dtype: int64

In [19]:
############################################################
"""Annotate each sample by its irAluType"""
############################################################
# Create masks for NA conditions
both_na = cass_df[['ui_alu_orient', 'di_alu_orient']].isna().all(axis=1)
ui_na = cass_df['ui_alu_orient'].isna() & cass_df['di_alu_orient'].notna()
di_na = cass_df['di_alu_orient'].isna() & cass_df['ui_alu_orient'].notna()

# Create masks for non-NA combinations
irAlu_cond = (
    (cass_df['ui_alu_orient'].str.contains('as', na=False)) & 
    (cass_df['di_alu_orient'].str.contains('ss', na=False))
) | (
    (cass_df['ui_alu_orient'].str.contains('ss', na=False)) & 
    (cass_df['di_alu_orient'].str.contains('as', na=False))
)

as_both_cond = (
    cass_df['ui_alu_orient'].str.contains('as', na=False) & 
    cass_df['di_alu_orient'].str.contains('as', na=False) &
    ~irAlu_cond
)

ss_both_cond = (
    cass_df['ui_alu_orient'].str.contains('ss', na=False) & 
    cass_df['di_alu_orient'].str.contains('ss', na=False) &
    ~irAlu_cond
)

# Define conditions and choices in priority order
conditions = [
    both_na,
    ui_na,
    di_na,
    irAlu_cond,
    as_both_cond,
    ss_both_cond
]

choices = [
    'no alu',
    'no alu ui',
    'no alu di',
    'irAlu',
    'as di and ui',
    'ss di and ui'
]

# Create new column with default 'other' for unhandled cases
cass_df['irAlu_type'] = np.select(conditions, choices, default='other')


#################################################
""" Calculate irAlu distance and get irAlu name"""
#################################################
new_cols = [
    'last_ui_alu_orient', 'last_ui_alu_start', 'last_ui_alu_end', 'last_ui_alu_name',
    'second_last_ui_alu_orient', 'second_last_ui_alu_end', 'second_last_ui_alu_name',
    'first_di_alu_orient', 'first_di_alu_start', 'first_di_alu_end', 'first_di_alu_name',
    'second_di_alu_orient', 'second_di_alu_start', 'second_di_alu_name',
    'alu_pair_dist', 'ui_alu_pair_dist', 'di_alu_pair_dist', 'alu_pair_name', 'alu_pair_type'
]

for col in new_cols:
    cass_df[col] = None

# Process only 'irAlu' rows
irAlu_mask = cass_df['irAlu_type'] == 'irAlu'

def process_irAlu(row):
    def to_list(val, keep_before_hash=False):
        if pd.isna(val):
            return []
        items = [v.strip() for v in str(val).split(',') if v.strip()]
        if keep_before_hash:
            return [v.split('#')[0] for v in items]
        return items

    # Extract lists from columns
    ui_orient = row['ui_alu_orient']
    ui_start = row['ui_alu_start']
    ui_end = row['ui_alu_end']
    ui_name = row['ui_alu_name']
    di_orient = row['di_alu_orient']
    di_start = row['di_alu_start']
    di_end = row['di_alu_end']
    di_name = row['di_alu_name']

    # Parse comma-separated values into lists
    ui_orient = to_list(row['ui_alu_orient'])
    ui_start = to_list(row['ui_alu_start'])
    ui_end = to_list(row['ui_alu_end'])
    ui_name = to_list(row['ui_alu_name'], keep_before_hash = True)
    di_orient = to_list(row['di_alu_orient'])
    di_start = to_list(row['di_alu_start'])
    di_end = to_list(row['di_alu_end'])
    di_name = to_list(row['di_alu_name'], keep_before_hash = True)

    # Convert to integers where appropriate
    ui_start = [int(x) for x in ui_start]
    ui_end = [int(x) for x in ui_end]
    di_start = [int(x) for x in di_start]
    di_end = [int(x) for x in di_end]
    
    result = {col: None for col in new_cols}
    
    # (1) Last UI
    if isinstance(ui_orient, list) and len(ui_orient) >= 1:
        result['last_ui_alu_orient'] = ui_orient[-1]
        result['last_ui_alu_start'] = ui_start[-1]
        result['last_ui_alu_end'] = ui_end[-1]
        result['last_ui_alu_name'] = ui_name[-1]

    # (2) Second last UI
    if isinstance(ui_orient, list) and len(ui_orient) >= 2:
        result['second_last_ui_alu_orient'] = ui_orient[-2]
        result['second_last_ui_alu_end'] = ui_end[-2]
        result['second_last_ui_alu_name'] = ui_name[-2]

    # (3) First DI
    if isinstance(di_orient, list) and len(di_orient) >= 1:
        result['first_di_alu_orient'] = di_orient[0]
        result['first_di_alu_start'] = di_start[0]
        result['first_di_alu_end'] = di_end[0]
        result['first_di_alu_name'] = di_name[0]

    # (4) Second DI
    if isinstance(di_orient, list) and len(di_orient) >= 2:
        result['second_di_alu_orient'] = di_orient[1]
        result['second_di_alu_start'] = di_start[1]
        result['second_di_alu_name'] = di_name[1]

    # (5) Distances
    if result['last_ui_alu_end'] is not None and result['first_di_alu_start'] is not None:
        result['alu_pair_dist'] = result['first_di_alu_start'] - result['last_ui_alu_end']

    if result['second_last_ui_alu_end'] is not None and result['last_ui_alu_start'] is not None:
        result['ui_alu_pair_dist'] = result['last_ui_alu_start'] - result['second_last_ui_alu_end']

    if result['first_di_alu_end'] is not None and result['second_di_alu_start'] is not None:
        result['di_alu_pair_dist'] = result['second_di_alu_start'] - result['first_di_alu_end']

    # (6) Names and Types
    if result['last_ui_alu_name'] and result['first_di_alu_name']:
        result['alu_pair_name'] = f"{result['last_ui_alu_name']}-{result['first_di_alu_name']}"

    if result['last_ui_alu_orient'] and result['first_di_alu_orient']:
        result['alu_pair_type'] = f"{result['last_ui_alu_orient']}-{result['first_di_alu_orient']}"
        
    return pd.Series(result)


# Apply processing only to irAlu rows
cass_df.loc[irAlu_mask, new_cols] = cass_df[irAlu_mask].apply(process_irAlu, axis=1)


############################################################
"""Divide irAlu samples into three subtypes"""
############################################################

# Now refine irAlu samples into three subcategories
irAlu_mask = cass_df['irAlu_type'] == 'irAlu'

# Condition 1: irAlu standard
second_last_ui_na = cass_df['second_last_ui_alu_orient'].isna()
second_di_na = cass_df['second_di_alu_orient'].isna()

# Standard irAlu condition (now properly handling NAs)
irAlu_standard = (
    (
        (cass_df['second_last_ui_alu_orient'].ne('as') | second_last_ui_na) & 
        (cass_df['last_ui_alu_orient'].eq('ss')) & 
        (cass_df['first_di_alu_orient'].eq('as')) & 
        (cass_df['second_di_alu_orient'].ne('ss') | second_di_na)
    ) | (
        (cass_df['second_last_ui_alu_orient'].ne('ss') | second_last_ui_na) & 
        (cass_df['last_ui_alu_orient'].eq('as')) & 
        (cass_df['first_di_alu_orient'].eq('ss')) & 
        (cass_df['second_di_alu_orient'].ne('as') | second_di_na)
    )
) & irAlu_mask

# Condition 2: irAlu with competition
irAlu_competition = (
    (
        (cass_df['last_ui_alu_orient'].eq('ss')) & 
        (cass_df['first_di_alu_orient'].eq('as')) & 
        (
            (cass_df['second_last_ui_alu_orient'].eq('as')) | 
            (cass_df['second_di_alu_orient'].eq('ss'))
        )
    ) | (
        (cass_df['last_ui_alu_orient'].eq('as')) & 
        (cass_df['first_di_alu_orient'].eq('ss')) & 
        (
            (cass_df['second_last_ui_alu_orient'].eq('ss')) | 
            (cass_df['second_di_alu_orient'].eq('as'))
        )
    )
) & irAlu_mask

# Condition 3: irAlu other type (remaining irAlu samples)
irAlu_other = irAlu_mask & ~irAlu_standard & ~irAlu_competition

# Update the classification
cass_df.loc[irAlu_standard, 'irAlu_type'] = 'irAlu standard'
cass_df.loc[irAlu_competition, 'irAlu_type'] = 'irAlu with competition'
cass_df.loc[irAlu_other, 'irAlu_type'] = 'irAlu other type'


# 1. Create the new column as a copy of 'alu_subtype'
cass_df['alu_subtype_by_dist'] = cass_df['alu_subtype']

# 2. Apply your specific conditions using np.where
cass_df['alu_subtype_by_dist'] = np.where(
    (cass_df['alu_type'] == 'irAlu') & (cass_df['irAlu_type'] == 'irAlu standard'),
    'irAlu standard',
    np.where(
        (cass_df['alu_type'] == 'irAlu') & (cass_df['irAlu_type'] == 'irAlu with competition'),
        'irAlu with competition',
        np.where(
            (cass_df['alu_type'] == 'irAlu') & (cass_df['irAlu_type'] == 'irAlu other type'),
            'irAlu other type',
            cass_df['alu_subtype_by_dist']
        )
    )
)

In [20]:
old_cass_result = pd.read_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.Merged.DiffAlterSplicing.results.step1.csv")
# Rename columns
old_cass_result.rename(
    columns={
        'alu_type': 'alu_type_old',
        'alu_subtype': 'alu_subtype_old',
        'alu_specific_subtype': 'alu_specific_subtype_old',
        'all_asAlu_ui': 'all_asAlu_ui_old',
        'all_ssAlu_ui': 'all_ssAlu_ui_old',
        'all_asAlu_exon': 'all_asAlu_exon_old',
        'all_ssAlu_exon': 'all_ssAlu_exon_old',
        'all_asAlu_di': 'all_asAlu_di_old',
        'all_ssAlu_di': 'all_ssAlu_di_old',
        'Ku_asAlu_ui': 'ku_asAlu_ui_old',
        'Ku_ssAlu_ui': 'ku_ssAlu_ui_old',
        'Ku_asAlu_exon': 'ku_asAlu_exon_old',
        'Ku_ssAlu_exon': 'ku_ssAlu_exon_old',
        'Ku_asAlu_di': 'ku_asAlu_di_old',
        'Ku_ssAlu_di': 'ku_ssAlu_di_old',
        'ku_bound': 'ku_bound_old'
    },
    inplace=True
)

# List of columns to merge from cass_df
columns_to_merge = [
    'alu_type', 'alu_subtype', 'alu_specific_subtype', 'irAlu_type','alu_pair_name', 'alu_pair_type',
    'all_asAlu_ui', 'all_ssAlu_ui', 'all_asAlu_exon', 'all_ssAlu_exon',
    'all_asAlu_di', 'all_ssAlu_di', 'ku_asAlu_ui', 'ku_ssAlu_ui','ui_alu_ku_tag_num', 
    'ku_asAlu_exon', 'ku_ssAlu_exon','exon_alu_ku_tag_num',
    'ku_asAlu_di', 'ku_ssAlu_di','di_alu_ku_tag_num','ui_alu_name', 'ui_alu_orient',
    'exon_alu_name', 'exon_alu_orient', 'di_alu_name', 'di_alu_orient',
    'alu_pair_dist', 'ui_alu_pair_dist', 'di_alu_pair_dist',
]

# Perform the merge (left join to preserve all rows in old_cass_result)
merged_cass_df = old_cass_result.merge(
    cass_df[['name'] + columns_to_merge],
    on='name',
    how='left'
)

In [21]:
"""Update the Ku_bound column with new Ku-Alu overlap information"""
merged_cass_df.rename(columns={'ku_bound': 'ku_bound_old'}, inplace=True)
print(merged_cass_df['ku_bound_old'].value_counts())

# Step 1: Create new column with default value 0
merged_cass_df['ku_bound'] = 0

# Step 2: Update for 'asAlu' type
mask_asAlu = (merged_cass_df['alu_type'] == 'asAlu') & (merged_cass_df['ku_asAlu_exon'] > 0)
merged_cass_df.loc[mask_asAlu, 'ku_bound'] = 1

# Step 3: Update for 'ssAlu' type
mask_ssAlu = (merged_cass_df['alu_type'] == 'ssAlu') & (merged_cass_df['ku_ssAlu_exon'] > 0)
merged_cass_df.loc[mask_ssAlu, 'ku_bound'] = 1

# Step 4: Update for 'irAlu' and 'other' types
mask_ir_other = merged_cass_df['alu_type'].isin(['irAlu', 'other'])
# Columns to check for positive values
check_cols = ['ku_asAlu_ui', 'ku_ssAlu_ui', 'ku_asAlu_di', 'ku_ssAlu_di']
# Check if any of the specified columns have value > 0
any_positive = merged_cass_df.loc[mask_ir_other, check_cols].gt(0).any(axis=1)
# Update ku_bound where conditions are met
merged_cass_df.loc[any_positive.index, 'ku_bound'] = any_positive.astype(int)
print(merged_cass_df['ku_bound'].value_counts())

ku_bound_old
0    21800
1    20961
Name: count, dtype: int64
ku_bound
0    39031
1     3730
Name: count, dtype: int64


In [22]:
merged_cass_df['ku_asAlu_ui'].value_counts()

ku_asAlu_ui
0     40245
1      1787
2       445
3       120
4        83
5        30
7        18
6        13
10       10
8         7
12        2
9         1
Name: count, dtype: int64

In [25]:
merged_cass_df['ku_asAlu_ui'].value_counts()

ku_asAlu_ui
0     27890
1      5768
2      2812
3      1603
4      1009
      ...  
54        1
84        1
59        1
49        1
57        1
Name: count, Length: 66, dtype: int64

In [24]:
#merged_cass_df.to_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DASE_Results/Cass.AluOverlap.071825.csv')
merged_cass_df.to_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.110325.csv')

In [ ]:
cass_df.rename(columns={'all_ssAlu_ui': 'all_ssAlu_ui.1', 'all_asAlu_ui': 'all_asAlu_ui.1',
                        'all_ssAlu_exon': 'all_ssAlu_exon.1', 'all_asAlu_exon': 'all_asAlu_exon.1',
                        'all_ssAlu_di': 'all_ssAlu_di.1', 'all_asAlu_di': 'all_asAlu_di.1',
                        'Ku_ssAlu_ui': 'Ku_ssAlu_ui.1', 'Ku_asAlu_ui': 'Ku_asAlu_ui.1',
                        'Ku_ssAlu_exon': 'Ku_ssAlu_exon.1', 'Ku_asAlu_exon': 'Ku_asAlu_exon.1',
                        'Ku_ssAlu_di': 'Ku_ssAlu_di.1', 'Ku_asAlu_di': 'Ku_asAlu_di.1',
                        'alu_type': 'alu_type.1', 'alu_subtype': 'alu_subtype.1', 'alu_specific_subtype': 'alu_specific_subtype.1'
                        }, inplace=True)

new_cass_df = pd.read_csv(os.path.join(input_dir, file_names['cass']))

### Merge Alu Tags to cass_df_with_gtex_brainspan_studer_psi

In [ ]:
# Please refer to 
merged_ku_overlap_alu_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_Ku_Overlap_Alu/Merged_Ku_Overlap_Alu.csv')
cass_df_with_gtex_brainspan_studer_psi_alu = merged_cass_df.merge(merged_ku_overlap_alu_df, on='name', how='left')
print(len(cass_df_with_gtex_brainspan_studer_psi_alu))

### Distribution of asAlu

In [17]:
"""Construct asAlu2exon data frame that stores exons containing asAlu"""

# Read Alu reference file
alu_columns = [
    'chrom', 'start', 'end', 'name', 'score', 'strand'
]

alu_ref_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/CLIP Data/Alu-CassExon Reference Data/rmsk.Alu.uniqName.bed', sep='\t', header=None, names=alu_columns)


# Read asAlu2exon file
asAlu2exon = pd.read_csv('/Users/tianji/Desktop/Alu Project/CLIP Data/Alu-map-all-CassExons/Alu.map.reverse_ss_cass.uniq.bed.annot.txt', sep='\t')

# Rename the '#name' column to 'name.orig'
asAlu2exon = asAlu2exon.rename(columns={"#name": "name.orig"})

# Create 'Alu_type' as substring before '#' in the 'name.orig' column 
asAlu2exon['Alu_type'] = asAlu2exon['name.orig'].str.split('#').str[0]

# Create 'Alu_id' as substring before '//' in the 'name.orig' column 
asAlu2exon['Alu_id'] = asAlu2exon['name.orig'].str.split('//').str[0]

# Create 'name' as substring after '//' in the 'name.orig' column 
asAlu2exon['name'] = asAlu2exon['name.orig'].str.split('//').str[1]

# Define conditions based on presence of substrings
conditions = [
    asAlu2exon['Alu_type'].str.contains('AluS'),
    asAlu2exon['Alu_type'].str.contains('AluY'),
    asAlu2exon['Alu_type'].str.contains('AluJ')
]

# Define corresponding values
choices = ['AluS', 'AluY', 'AluJ']

# Create the new 'Alu_family' column
asAlu2exon['Alu_family'] = np.select(conditions, choices, default='Other')

""" Read in the cass_df, we need to merged the "alu_type", "filter(Cyc.HCT116.KuKD)", and "filter(G1Arr.HCT116.KuKD)" columns to asAlu2exon 
then use "filter(Cyc.HCT116.KuKD)" and "filter(G1Arr.HCT116.KuKD)" information to determine whether a sample is regulated by Ku"""
cass_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DASE_Results/Cass.Merged.DiffAlterSplicing.results.csv')

# Step 1: Merge selected columns from cass_df into asAlu2exon by 'name'
asAlu2exon = asAlu2exon.merge(
    cass_df[['name', 'alu_type', 'filter(Cyc.HCT116.KuKD)', 'filter(G1Arr.HCT116.KuKD)']],
    on='name',
    how='left'
)

# Step 2: Define ku_regulated based on filters being -1 or 1
asAlu2exon['ku_regulated'] = (
    asAlu2exon['filter(Cyc.HCT116.KuKD)'].isin([-1, 1]) |
    asAlu2exon['filter(G1Arr.HCT116.KuKD)'].isin([-1, 1])
)

# Step 3: Drop the 'name.orig' column
asAlu2exon = asAlu2exon.drop(columns=['name.orig'])

# Step 4–5: Group by 'name' and merge 'Alu_type' and 'Alu_id' to handle samples containing multiple asAlu elements
asAlu2exon_merged = (
    asAlu2exon
    .groupby('name', as_index=False)
    .agg({
        'gene_id': 'first',
        'gene_symbol': 'first',
        'region': 'first',
        'Alu_type': lambda x: ','.join(sorted(set(x))),
        'Alu_id': lambda x: ','.join(sorted(set(x))),
        'Alu_family': lambda x: ','.join(sorted(set(x))),
        'ku_regulated': 'first'
    })
)

In [3]:
print(len(asAlu2exon))
print(len(asAlu2exon_merged))

2704
2678


In [4]:
print(asAlu2exon_merged['Alu_family'].value_counts())
#asAlu2exon_merged.to_csv('/Users/tianji/Desktop/Alu Project/Alu_Family_Analysis/asAlu.Overlap.CassMiddleExon.csv', index = False)

Alu_family
AluS         1439
AluJ         1048
AluY          181
AluJ,AluS       6
AluS,AluY       4
Name: count, dtype: int64


In [20]:
print(asAlu2exon_merged[asAlu2exon_merged['ku_regulated'] == True]['Alu_family'].value_counts())

Alu_family
AluS         48
AluJ         19
AluY          2
AluS,AluY     1
Name: count, dtype: int64


In [14]:
import pandas as pd

# 1) Exonized asAlu
type_counts = asAlu2exon['Alu_type'].value_counts()
type_percentages = type_counts / type_counts.sum() * 100

percent_family_total_exonized = pd.concat([type_counts, type_percentages], axis=1).reset_index()
percent_family_total_exonized.columns = ['Alu_type', 'Count', 'Percentage']

# 2) Exonized + Ku‑regulated asAlu
ku_regulated_exon = asAlu2exon[asAlu2exon['ku_regulated'] == True]
type_counts_ku = ku_regulated_exon['Alu_type'].value_counts()
type_percentages_ku = type_counts_ku / type_counts_ku.sum() * 100

percent_family_total_ku = pd.concat([type_counts_ku, type_percentages_ku], axis=1).reset_index()
percent_family_total_ku.columns = ['Alu_type', 'Count', 'Percentage']

# 3) All Alu in the genome
alu_ref_df['Alu_type'] = alu_ref_df['name'].str.split('#').str[0]
type_counts_all = alu_ref_df['Alu_type'].value_counts()
type_percentages_all = type_counts_all / type_counts_all.sum() * 100

percent_family_total_asalu = pd.concat([type_counts_all, type_percentages_all], axis=1).reset_index()
percent_family_total_asalu.columns = ['Alu_type', 'Count', 'Percentage']




In [ ]:
# 1) Rename columns for clarity
percent_family_total_asalu = percent_family_total_asalu.rename(
    columns={'Count': 'Count_total_asAlu', 'Percentage': 'Pct_total_asAlu'}
)
percent_family_total_exonized = percent_family_total_exonized.rename(
    columns={'Count': 'Count_exonized_asAlu', 'Percentage': 'Pct_exonized_asAlu'}
)
percent_family_total_ku = percent_family_total_ku.rename(
    columns={'Count': 'Count_ku_regulated_asAlu', 'Percentage': 'Pct_ku_regulated_asAlu'}
)

# 2) Fill missing types in 'exonized' to match genome-wide set
missing_ex = set(percent_family_total_asalu['Alu_type']) - set(percent_family_total_exonized['Alu_type'])
for t in missing_ex:
    percent_family_total_exonized = pd.concat([
        percent_family_total_exonized,
        pd.DataFrame([{
            'Alu_type': t,
            'Count_exonized_asAlu': 0,
            'Pct_exonized_asAlu': 0
        }])
    ], ignore_index=True)

# 3) Fill missing types in 'ku' to match genome-wide set
missing_ku = set(percent_family_total_asalu['Alu_type']) - set(percent_family_total_ku['Alu_type'])
for t in missing_ku:
    percent_family_total_ku = pd.concat([
        percent_family_total_ku,
        pd.DataFrame([{
            'Alu_type': t,
            'Count_ku_regulated_asAlu': 0,
            'Pct_ku_regulated_asAlu': 0
        }])
    ], ignore_index=True)

# 4) Merge genome-wide vs exonized (outer join to keep all types)
merged_alu_type_df = percent_family_total_asalu.merge(
    percent_family_total_exonized,
    on='Alu_type',
    how='outer'
)

# 5) Merge in Ku-regulated summary
merged_alu_type_df = merged_alu_type_df.merge(
    percent_family_total_ku,
    on='Alu_type',
    how='left'
)

# 6) Compute distance‑from‑diagonal metrics
merged_alu_type_df['Distance_genome_vs_exonized'] = (
    (merged_alu_type_df['Pct_exonized_asAlu'] - merged_alu_type_df['Pct_total_asAlu']).abs()
    / np.sqrt(2)
)
merged_alu_type_df['Distance_exonized_vs_ku'] = (
    (merged_alu_type_df['Pct_ku_regulated_asAlu'] - merged_alu_type_df['Pct_exonized_asAlu']).abs()
    / np.sqrt(2)
)

#merged_alu_type_df.to_csv('/Users/tianji/Desktop/Alu Project/Alu_Family_Analysis/Percent_Alu_Family_Summary.csv', index=False)



In [16]:
merged_alu_type_df.head()

,Alu_type,Count_total_asAlu,Pct_total_asAlu,Count_exonized_asAlu,Pct_exonized_asAlu,Count_ku_regulated_asAlu,Pct_ku_regulated_asAlu,Distance_genome_vs_exonized,Distance_exonized_vs_ku
0,AluJb,144887,12.688939,528,19.526627,10,13.888889,4.834976,3.986483
1,AluJo,72247,6.327260,223,8.247041,5,6.944444,1.357490,0.921075
2,AluJr,77181,6.759371,247,9.134615,3,4.166667,1.679551,3.512870
3,AluJr4,17674,1.547857,58,2.144970,1,1.388889,0.422223,0.534630
4,AluSc,34506,3.021972,86,3.180473,0,0.000000,0.112077,2.248934


### Old code for Alu overlap

In [ ]:
def calculate_overlaps(cass_exon_df, exon_coordinates, alu_coordinates, min_overlap=0):
    # Create a copy of the original DataFrame
    reverse_strand_alu_coordinates = alu_coordinates.copy()

    # Reverse the strand values
    reverse_strand_alu_coordinates['strand'] = reverse_strand_alu_coordinates['strand'].map({'+': '-', '-': '+'})

    chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 
              'chr21', 'chr22', 'chrX', 'chrY']
    strands = ['+','-']
    cass_df = cass_exon_df.copy()
    
    for strand in strands:
        for chrom in chroms:
            print(f"Processing {strand} strand in {chrom}")
            sorted_exon_df = exon_coordinates[(exon_coordinates['chrom'] == chrom) & (exon_coordinates['strand'] == strand)]
            sorted_alu_ref = alu_coordinates[(alu_coordinates['chrom'] == chrom) & (alu_coordinates['strand'] == strand)]
            sorted_reverse_strand_alu_ref = reverse_strand_alu_coordinates[(reverse_strand_alu_coordinates['chrom'] == chrom) & (reverse_strand_alu_coordinates['strand'] == strand)]
            overlap_result = irAlu_overlaps(sorted_exon_df, sorted_alu_ref, sorted_reverse_strand_alu_ref, min_overlap)

            # Merge overlap_result to cass_df
            missing_names = set(overlap_result['name_exon']) - set(cass_df['name'])
            if missing_names:
                raise ValueError(f"Some 'name_exon' values not found in cass_df['name']: {missing_names}")

            # Step 2: Subset and rename for merging
            cols_to_merge = [
                'name_exon',  # Use exon name as key
                'start_alu', 'end_alu', 'name_alu', 'score', 'alu_index',
                'dist_to_ui_start', 'dist_to_ui_end', 'alu_orient'
            ]
            overlap_subset = overlap_result[cols_to_merge].copy()
            overlap_subset = overlap_subset.rename(columns={'name_exon': 'name'})  # Rename to match cass_df

            # Step 3: Merge — will overwrite previous values for overlapping names
            cass_df = pd.merge(cass_df, overlap_subset, on='name', how='left')

    return cass_df